# Imports

In [1]:
import numpy as np
import pandas as pd
import random

# Funcs

In [2]:
mem = {}

def encode_board(num_board):
    enc = {1 : 'x', 0 : '-', -1 : 'o'}
    return ''.join(enc[num_board[i // 3][i % 3]] for i in range(9))

def decode_board(sym_board):
    board = np.zeros([3, 3]).astype(float)
    for i in range(3):
        for j in range(3):
            if sym_board[3 * i + j] == 'x':
                board[i][j] = 1
            elif sym_board[3 * i + j] == 'o':
                board[i][j] = -1
    return board

def check_win(board, turn):
    if board[0][0] + board[1][0] + board[2][0] == 3 * turn:
        return True
    if board[0][1] + board[1][1] + board[2][1] == 3 * turn:
        return True
    if board[0][2] + board[1][2] + board[2][2] == 3 * turn:
        return True
    if board[0][0] + board[0][1] + board[0][2] == 3 * turn:
        return True
    if board[1][0] + board[1][1] + board[1][2] == 3 * turn:
        return True
    if board[2][0] + board[2][1] + board[2][2] == 3 * turn:
        return True
    if board[0][0] + board[1][1] + board[2][2] == 3 * turn:
        return True
    if board[0][2] + board[1][1] + board[2][0] == 3 * turn:
        return True
    return False

def pure_search(board):
    hashed = encode_board(board)
    if hashed in mem:
        return mem[hashed]
    turn = 1 - board.sum() * 2
    if check_win(board, turn):
        mem[hashed] = [1, [-1, -1]]
        return [1, [-1, -1]]
    elif check_win(board, -turn):
        mem[hashed] = [-1, [-1, -1]]
        return [-1, [-1, -1]]
    elif (board == 0).sum() == 0:
        mem[hashed] = [0, [-1, -1]]
        return [0, [-1, -1]]
    legal_moves = [[i // 3, i % 3] for i in range(9) if board[i // 3][i % 3] == 0]
    evals = []
    for move in legal_moves:
        board[move[0]][move[1]] = turn
        evals.append([-pure_search(board)[0], move])
        board[move[0]][move[1]] = 0
    evals.sort()
    mem[hashed] = evals[-1]
    return evals[-1]

In [3]:
pure_search(np.zeros([3, 3]))

[0, [2, 2]]

# Save

In [4]:
d = {'position': mem.keys(), 'eval': list(mem[i][0] for i in mem.keys())}
df = pd.DataFrame(data = d).reset_index(drop = True)
df.to_csv("db.csv", index = False)